In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
terminología= "/content/drive/My Drive/Seguimiento-Lucía/terminología/enlazado_llm/labourlawterminologyv1.nt"
eurovoc="/content/drive/My Drive/Seguimiento-Lucía/papers/ijhac/data/eurovoc-skos-ap-eu.rdf"

In [ ]:
#extraer tripletas cuyo objeto es una uri de eurovoc
import requests

def extraer_tripletas_eurovoc(ruta_archivo_nt):
    tripletas_eurovoc = []

    with open(ruta_archivo_nt, "r", encoding="utf-8") as f:
        for linea in f:
            linea = linea.strip()
            if not linea or not linea.endswith(" ."):
                continue

            partes = linea.split(" ", 2)
            if len(partes) < 3:
                continue

            sujeto, predicado, resto = partes
            objeto = resto.rsplit(" ", 1)[0]  # elimina el punto final

            if objeto.startswith("<http://eurovoc.europa.eu/"):
                tripletas_eurovoc.append((sujeto, predicado, objeto))

    return tripletas_eurovoc

def guardar_tripletas_nt(tripletas, archivo_salida):
    with open(archivo_salida, "w", encoding="utf-8") as f:
        for s, p, o in tripletas:
            f.write(f"{s} {p} {o} .\n")

def obtener_literal_eurovoc(uri, lang="es"):
    headers = {
        "Accept": "application/json"
    }
    response = requests.get(uri, headers=headers)

    if response.status_code == 200:
        data = response.json()
        labels = data.get("skos:prefLabel", [])
        for label in labels:
            if label.get("lang") == lang:
                return label.get("value")
    return None

def convertir_uris_a_literales(tripletas, lang="es"):
    tripletas_literales = []

    for s, p, o in tripletas:
        uri = o.strip("<>")
        literal = obtener_literal_eurovoc(uri, lang=lang)
        if literal:
            # Escribimos el literal como un string con idioma
            tripletas_literales.append((s, p, f"\"{literal}\"@{lang}"))
        else:
            # Si no encontramos literal, mantenemos la URI
            tripletas_literales.append((s, p, o))

    return tripletas_literales


In [ ]:
import re

def cargar_prefLabels_eurovoc(ruta_dump):
    etiquetas = {}
    with open(ruta_dump, "r", encoding="utf-8") as f:
        contenido = f.read()

    bloques = re.findall(r'<rdf:Description rdf:about="([^"]+)">(.*?)</rdf:Description>', contenido, re.DOTALL)
    for uri, bloque in bloques:
        match = re.search(r'<skos:prefLabel[^>]*xml:lang="es">(.+?)</skos:prefLabel>', bloque)
        if match:
            etiquetas[f"<{uri}>"] = f"\"{match.group(1)}\"@es"
    return etiquetas

def cargar_literal_lynx_desde_nt(ruta_nt):
    etiquetas = {}
    with open(ruta_nt, "r", encoding="utf-8") as f:
        for linea in f:
            linea = linea.strip()
            if not linea.endswith(" ."):
                continue
            partes = linea.split(" ", 2)
            if len(partes) < 3:
                continue
            sujeto, predicado, resto = partes
            objeto = resto.rsplit(" ", 1)[0]
            if sujeto.startswith("<http://lynx-project.eu/") and objeto.endswith("@es"):
                etiquetas[sujeto] = objeto
    return etiquetas

def extraer_tripletas_objeto_eurovoc(ruta_nt):
    tripletas = []
    with open(ruta_nt, "r", encoding="utf-8") as f:
        for linea in f:
            linea = linea.strip()
            if not linea.endswith(" ."):
                continue
            partes = linea.split(" ", 2)
            if len(partes) < 3:
                continue
            sujeto, predicado, resto = partes
            objeto = resto.rsplit(" ", 1)[0]
            if objeto.startswith("<http://eurovoc.europa.eu/"):
                tripletas.append((sujeto, predicado, objeto))
    return tripletas

def guardar_tripletas_nt(tripletas, archivo_salida):
    with open(archivo_salida, "w", encoding="utf-8") as f:
        for s, p, o in tripletas:
            f.write(f"{s} {p} {o} .\n")

def convertir_tripletas_a_literales(tripletas, etiquetas_eurovoc, etiquetas_lynx):
    tripletas_convertidas = []
    for s, p, o in tripletas:
        if o in etiquetas_eurovoc:
            tripletas_convertidas.append((s, p, etiquetas_eurovoc[o]))
        elif o in etiquetas_lynx:
            tripletas_convertidas.append((s, p, etiquetas_lynx[o]))
        else:
            tripletas_convertidas.append((s, p, o))  # Se conserva la URI si no hay literal
    return tripletas_convertidas


In [ ]:
eurovoc_labels = cargar_prefLabels_eurovoc(eurovoc)
lynx_labels = cargar_literal_lynx_desde_nt(terminología)
# Paso 1: Extraer tripletas cuyo objeto es URI de Eurovoc
tripletas_uris = extraer_tripletas_objeto_eurovoc(terminología)
guardar_tripletas_nt(tripletas_uris, "/content/drive/My Drive/Seguimiento-Lucía/terminología/enlazado_llm/tripletas_uris.nt")

# Paso 2: Sustituir URIs por literales
tripletas_literales = convertir_tripletas_a_literales(tripletas_uris, eurovoc_labels, lynx_labels)
guardar_tripletas_nt(tripletas_literales, "/content/drive/My Drive/Seguimiento-Lucía/terminología/enlazado_llm/tripletas_convertidas.nt")

In [ ]:
!pip install rdflib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 565.1/565.1 kB 9.3 MB/s eta 0:00:00


In [ ]:
from rdflib import Graph, URIRef, Namespace
import re

# Cargar etiquetas desde archivo .nt (LYNX)
def extraer_etiquetas_nt(archivo):
    g = Graph()
    g.parse(archivo, format='nt')
    etiquetas = {}
    for s, p, o in g:
        if isinstance(s, URIRef) and hasattr(o, 'language') and o.language == 'es':
            etiquetas[str(s)] = str(o)
    return etiquetas

# Cargar etiquetas desde archivo .rdf (EUROVOC)
def extraer_etiquetas_rdf(archivo):
    g = Graph()
    g.parse(archivo, format='xml')

    SKOSXL = Namespace("http://www.w3.org/2008/05/skos-xl#")

    literal_forms = {
        str(label): str(literal)
        for label, _, literal in g.triples((None, SKOSXL.literalForm, None))
        if hasattr(literal, 'language') and literal.language == 'es'
    }

    etiquetas = {}
    for concept, _, label_uri in g.triples((None, SKOSXL.prefLabel, None)):
        if str(label_uri) in literal_forms:
            etiquetas[str(concept)] = literal_forms[str(label_uri)]

    return etiquetas

# Reemplazar URIs por etiquetas literales usando Lynx para sujetos y Eurovoc para objetos
def sustituir_uris_por_etiquetas(archivo_entrada, archivo_salida, etiquetas_sujetos, etiquetas_objetos):
    with open(archivo_entrada, 'r', encoding='utf-8') as f:
        lineas = f.readlines()

    nuevas_lineas = []
    for linea in lineas:
      partes = re.findall(r'<([^>]+)>', linea)
    if len(partes) == 3:
        sujeto_uri, predicado_uri, objeto_uri = partes
        sujeto = f"\"{etiquetas_sujetos.get(sujeto_uri, sujeto_uri)}\"" if sujeto_uri in etiquetas_sujetos else f"<{sujeto_uri}>"
        predicado = f"<{predicado_uri}>"  # no se reemplaza
        objeto = f"\"{etiquetas_objetos.get(objeto_uri, objeto_uri)}\"" if objeto_uri in etiquetas_objetos else f"<{objeto_uri}>"
        nueva_linea = f"{sujeto} {predicado} {objeto} .\n"
        nuevas_lineas.append(nueva_linea)
    else:
        nuevas_lineas.append(linea)  # mantener líneas que no son tripletas simples


    with open(archivo_salida, 'w', encoding='utf-8') as f:
        f.writelines(nuevas_lineas)

# Rutas a tus archivos
archivo_lynx = terminología
archivo_eurovoc = eurovoc
archivo_tripletas = "/content/drive/My Drive/Seguimiento-Lucía/terminología/enlazado_llm/tripletas_uris.nt"
archivo_salida = "/content/drive/My Drive/Seguimiento-Lucía/terminología/enlazado_llm/tripletas_convertidas.nt"

# Ejecutar proceso
etiquetas_lynx = extraer_etiquetas_nt(archivo_lynx)
etiquetas_eurovoc = extraer_etiquetas_rdf(archivo_eurovoc)

# Sustituir en archivo respetando origen de etiquetas
sustituir_uris_por_etiquetas(archivo_tripletas, archivo_salida, etiquetas_lynx, etiquetas_eurovoc)


In [ ]:
!pip install rdflib pandas requests

In [ ]:
import rdflib
import pandas as pd
import re

# Función para extraer prefLabels de Eurovoc desde RDF
def extraer_pref_labels_eurovoc(rdf_file):
    g = rdflib.Graph()
    g.parse(rdf_file, format='xml')

    pref_labels = {}
    for s, p, o in g.triples((None, rdflib.URIRef("http://www.w3.org/2004/02/skos/core#prefLabel"), None)):
        if isinstance(o, rdflib.Literal) and o.language == "es":
            pref_labels[str(s)] = str(o)

    return pref_labels

# Función para cargar las etiquetas de Lynx desde el archivo NT
def cargar_literal_lynx_desde_nt(ruta_nt):
    g = rdflib.Graph()
    g.parse(ruta_nt, format="nt")

    lynx_labels = {}
    for s, p, o in g.triples((None, rdflib.URIRef("http://www.w3.org/2004/02/skos/core#prefLabel"), None)):
        if isinstance(o, rdflib.Literal) and o.language == "es":
            lynx_labels[str(s)] = str(o)
    return lynx_labels

# Función para extraer las tripletas donde el objeto sea una URI de Eurovoc
def extraer_tripletas_objeto_eurovoc(ruta_nt):
    g = rdflib.Graph()
    g.parse(ruta_nt, format="nt")

    tripletas_eurovoc = []
    eurovoc_uri_pattern = re.compile(r"(eurovoc\.europa\.eu)")

    for s, p, o in g:
        if isinstance(o, rdflib.URIRef) and eurovoc_uri_pattern.search(str(o)):
            tripletas_eurovoc.append((str(s), str(p), str(o)))
    return tripletas_eurovoc

# Función para convertir las URIs en literales usando los diccionarios de etiquetas
def convertir_tripletas_a_literales(tripletas, eurovoc_labels, lynx_labels):
    tripletas_literales = []

    for s, p, o in tripletas:
        # Convertir sujeto y objeto a Literal si se encuentran en los diccionarios de etiquetas
        sujeto_literal = lynx_labels.get(s)
        objeto_literal = eurovoc_labels.get(o)

        # Verificar que ambos literales existen
        if sujeto_literal and objeto_literal:
            # Convertimos el sujeto y objeto en Literales y el predicado en URIRef
            tripletas_literales.append((rdflib.Literal(sujeto_literal, lang="es"), rdflib.URIRef(p), rdflib.Literal(objeto_literal, lang="es")))

    return tripletas_literales

# Función para guardar las tripletas en formato NT
def guardar_tripletas_nt(tripletas, ruta_salida):
    g = rdflib.Graph()
    for s, p, o in tripletas:
        # Convertir explícitamente los elementos a URIRef o Literal antes de agregar al grafo
        s = rdflib.URIRef(s) if isinstance(s, str) and s.startswith("http") else rdflib.Literal(s)
        p = rdflib.URIRef(p) if isinstance(p, str) and p.startswith("http") else rdflib.Literal(p)
        o = rdflib.URIRef(o) if isinstance(o, str) and o.startswith("http") else rdflib.Literal(o)
        g.add((s, p, o))

    g.serialize(destination=ruta_salida, format="nt", encoding="utf-8")

# Función principal
def procesar_tripletas(terminología, eurovoc_rdf, salida_tripletas_uris, salida_tripletas_convertidas):
    # Extraer las etiquetas de Eurovoc y Lynx
    eurovoc_labels = extraer_pref_labels_eurovoc(eurovoc_rdf)
    lynx_labels = cargar_literal_lynx_desde_nt(terminología)

    # Extraer tripletas que relacionan Lynx con Eurovoc
    tripletas_uris = extraer_tripletas_objeto_eurovoc(terminología)

    # Guardar las tripletas originales con URIs
    guardar_tripletas_nt(tripletas_uris, salida_tripletas_uris)

    # Convertir las tripletas a literales
    tripletas_literales = convertir_tripletas_a_literales(tripletas_uris, eurovoc_labels, lynx_labels)

    # Guardar las tripletas convertidas a literales
    guardar_tripletas_nt(tripletas_literales, salida_tripletas_convertidas)

# Rutas de archivos
terminología = "/content/drive/My Drive/Seguimiento-Lucía/terminología/enlazado_llm/labourlawterminologyv1.nt"
eurovoc_rdf = "/content/drive/My Drive/Seguimiento-Lucía/papers/ijhac/data/eurovoc-skos-ap-eu.rdf"
salida_tripletas_uris = "/content/drive/My Drive/Seguimiento-Lucía/terminología/enlazado_llm/tripletas_uris.nt"
salida_tripletas_convertidas = "/content/drive/My Drive/Seguimiento-Lucía/terminología/enlazado_llm/tripletas_convertidas.nt"

# Ejecutar el procesamiento
procesar_tripletas(terminología, eurovoc_rdf, salida_tripletas_uris, salida_tripletas_convertidas)

print("Proceso completado. Las tripletas se han guardado en los archivos correspondientes.")


Proceso completado. Las tripletas se han guardado en los archivos correspondientes.


In [ ]:
import re

# Ruta del archivo de entrada
archivo_entrada = "/content/drive/My Drive/Seguimiento-Lucía/terminología/enlazado_llm/tripletas_convertidas.nt"

# Archivos de salida
archivo_labour = "/content/drive/My Drive/Seguimiento-Lucía/terminología/enlazado_llm/labour_law_literales.txt"
archivo_eurovoc = "/content/drive/My Drive/Seguimiento-Lucía/terminología/enlazado_llm/eurovoc_literales.txt"

# Expresión regular para extraer el texto dentro de comillas antes de @es
def extraer_literal(literal):
    match = re.match(r'^"(.+?)"@es$', literal.strip())
    return match.group(1) if match else literal.strip()

labour_law_literales = []
eurovoc_literales = []

with open(archivo_entrada, "r", encoding="utf-8") as f:
    for linea in f:
        partes = linea.strip().split(" <http://www.w3.org/2004/02/skos/core#exactMatch> ")
        if len(partes) == 2:
            literal1 = extraer_literal(partes[0])
            literal2 = extraer_literal(partes[1].rstrip(" ."))
            labour_law_literales.append(literal1)
            eurovoc_literales.append(literal2)

# Guardar los resultados en archivos
with open(archivo_labour, "w", encoding="utf-8") as f:
    for item in labour_law_literales:
        f.write(item + "\n")

with open(archivo_eurovoc, "w", encoding="utf-8") as f:
    for item in eurovoc_literales:
        f.write(item + "\n")
